# LLM Email Autowriter Prototype

This notebook contains the original prototype code extracted from the working implementation.
The modular application in the `app/` directory is based on this code.

In [ ]:
from vllm import LLM, SamplingParams
import torch

# Model loading
model_name = "Qwen/Qwen2.5-1.5B-Instruct-GPTQ-Int8"
llm = LLM(model=model_name, dtype="auto")

In [ ]:
class ModelCall():
    def __init__(self, temperature:float=0.65, top_p:float=0.95, max_tokens:int=1024):
        self.temperature=temperature
        self.top_p=top_p
        self.max_tokens=max_tokens

    def makeQuery(self, query):
        from vllm import LLM, SamplingParams
        self.sampling_params = SamplingParams(temperature=self.temperature, top_p=self.top_p, max_tokens=self.max_tokens)
        if not any(word in query.lower() for word in ["email", "mail", "send an email", "write an email", "compose email"]):
            self.messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a strict assistant that only helps with writing emails. "
                        "If a user gives a prompt that is not related to writing an email, "
                        "you will ask them to provide a clear instruction to write an email."
                    )
                },
                {
                    "role": "user",
                    "content": query
                }
            ]
        else:
            self.messages = [
                {
                    "role": "system",
                    "content": (
                        "You are a helpful email writing assistant. "
                        "You are going to write an email based on the user's prompt. "
                        "You will write the email in a professional tone."
                    )
                },
                {
                    "role": "user",
                    "content": query
                }
            ]
    def retQuery(self):
        outputs = llm.chat(self.messages, self.sampling_params)
        return outputs[0].outputs[0].text

In [ ]:
# Usage example
llmQNA = ModelCall()
llmQNA.makeQuery("Write a professional email to my professor requesting a deadline extension for my assignment. Be polite, concise, and include that I've had unexpected personal issues.")
print(llmQNA.retQuery())